<h1 align="center">Deep Learning - Master in Deep Learning of UPM</h1> 

**IMPORTANTE**

Antes de empezar debemos instalar PyTorch Lightning, por defecto, esto valdría:

In [ ]:
!pip install pytorch-lightning

Además, si te encuentras ejecutando este código en Google Collab, lo mejor será que montes tu drive para tener acceso a los datos:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

En este ejercicio práctico vamos a utilizar los conocimientos adquiridos para abordar un caso de Regresión mediante PyTorch Lightning

In [ ]:
import pandas as pd

DATA_PATH = 'data/exercise.csv' # Pon tu ruta dependiendo de donde tengas el archivo en el Drive

df = pd.read_csv(DATA_PATH)

df.head() # Imprimamos las primeras filas del dataframe

# Dataset

In [ ]:
import torch
import pandas as pd

class RegressionDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ...

    def __len__(self):
        ...

    def __getitem__(self, idx):
        ...
        return features, target

# DataModule

In [ ]:
from sklearn.model_selection import train_test_split

# NO TOCAR
def split_train_val_test(df, val_size=0.2, test_size=0.2):
    eval_size = val_size + test_size # eval es un split intermedio que luego se divide en val y test
    test_prop = test_size / eval_size # proporción de test respecto a eval

    train, eval_ = train_test_split(df, test_size=eval_size)
    val, test = train_test_split(eval_, test_size=test_prop)
    return train, val, test

El DataModule debe recibir dinámicamente:
- El batch_size, por defecto a 64
- El número de trabajadores (num_workers), por defecto a 4
- El prefetch_factor, por defecto a 2
- Y el pin_memory, por defecto a True

Con ello deberá inicializar los dataloaders para entrenamiento, validación y test

In [ ]:
import pytorch_lightning
import numpy as np
from torch.utils.data import DataLoader

class RegressionDataModule(pytorch_lightning.LightningDataModule):
    def __init__(self, df, ...):
        super().__init__()
        ...

    def setup(self, stage=None): # esta función la ejecuta el trainer cuando se va a ejecutar el fit o el predict
        ...

    def collate_fn(self, batch): # PISTA: recordad que esto es regresión...
        ...

    def train_dataloader(self):
        ...

    def val_dataloader(self):
        ...

    def test_dataloader(self):
        ...

# LightningModule

La red debe ser un MLP a vuestra elección, también el optimizador.

Sin embargo, la función de pérdida se deberá adecuar para el caso de regresión...

La métrica que se deberá guardar será el R2 (disponible en [torchmetrics](https://lightning.ai/docs/torchmetrics/stable/regression/r2_score.html))

Ya os habréis fijado en que los steps de entrenamiento y validación son bastante semejante. Para ahorrar código, vamos a abstraer la parte común a otra función compute_batch() que recibirá el batch y el split para el que se realiza el step. Esta es la magia de Lightning, esta función no es nativa del LightningModule, es cosecha propia.

In [ ]:
from torchmetrics import R2Score
import torch.nn as nn

class Regressor(pytorch_lightning.LightningModule):
    def __init__(self, input_shape):
        super().__init__()

        # Inicializamos las capas de la red
        ...

        # Función de pérdida
        self.criterion = ...

        # Inicializamos las métricas
        self.r2 = R2Score()

    # Función forward como en un nn.Module de PyTorch
    def forward(self, x):
        ...
        return x
    
    
    def compute_batch(self, batch, split='train'):
        ...
        
        return loss
    
    def training_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'train')
    
    def validation_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'val')
    
    def test_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'test')
    
    def configure_optimizers(self):
        return ...

# Callbacks, Loggers y Trainer

**Ejercicio extra** - Implementar un callback que haga de timer. Este tendrá que registrar el momento en el que empieza el entrenamiento e imprimir el tiempo que ha transcurrido cuando este termina

In [ ]:
import pytorch_lightning
import time

class Timer(pytorch_lightning.Callback):
    ...

In [ ]:
import os
import datetime

SAVE_DIR = f'lightning_logs/exercise/{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'


# DataModule (NO TOCAR)
data = pd.read_csv(DATA_PATH)
data_module = RegressionDataModule(data, batch_size=64)

# LightningModule
input_shape = ... # Encontrar una manera dinámica de obtener el input_shape, PISTA: df.shape
model = Regressor(input_shape=input_shape)

# Callbacks
# Se deberá monitorizar la métrica 'val_r2' y guardar únicamente el mejor modelo.
early_stopping_callback = pytorch_lightning.callbacks.EarlyStopping(
    ...
)
model_checkpoint_callback = pytorch_lightning.callbacks.ModelCheckpoint(
    ...
)

# Descomentar las dos líneas siguientes si se ha implementado el callback Timer
# timer_callback = Timer()
# callbacks = [early_stopping_callback, model_checkpoint_callback, timer_callback]

callbacks = [early_stopping_callback, model_checkpoint_callback]

# Loggers (NO TOCAR)
csv_logger = pytorch_lightning.loggers.CSVLogger(
    save_dir=SAVE_DIR,
    name='metrics',
    version=None
)

loggers = [csv_logger] # se pueden poner varios loggers (mirar documentación)

# Trainer (NO TOCAR)
trainer = pytorch_lightning.Trainer(max_epochs=50, accelerator='gpu', devices=[0], callbacks=callbacks, logger=loggers)

trainer.fit(model, data_module)
results = trainer.test(model, data_module)

# Inferencia

In [ ]:
test_sample = data_module.test_df.sample(10)

inputs = torch.tensor(test_sample.drop('target', axis=1).values, dtype=torch.float32)
targets = torch.tensor(test_sample['target'].values, dtype=torch.float32)

model.eval()

with torch.no_grad():
    outputs = model(inputs)
    preds = outputs.squeeze().numpy()

for i, (pred, target) in enumerate(zip(preds, targets)):
    print(f"Predicción {i}: {pred:.2f}, Valor real: {target:.2f}")